# TDS LeNET Code Sample
The following code was pulled from this TDS article: https://towardsdatascience.com/implementing-yann-lecuns-lenet-5-in-pytorch-5e05a0911320

It details the architecture of LeNet5, and offers a sample pytorch implementation for the MNIST single-digit dataset. We modify this code to create a working LeNET implementation for our assignment dataset in the kaggle competetion.

### Helper functions

In [51]:
import numpy as np
from datetime import datetime 

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from torchvision import datasets, transforms

import matplotlib.pyplot as plt

# check device
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# model parameters
RANDOM_SEED = 42
LEARNING_RATE = 0.001
BATCH_SIZE = 32
N_EPOCHS = 15

IMG_SIZE = 32
N_CLASSES = 10

# calculate model accuracy
def get_accuracy(model, data_loader, device):
    '''
    Function for computing the accuracy of the predictions over the entire data_loader
    '''
    
    correct_pred = 0 
    n = 0
    
    with torch.no_grad():
        model.eval()
        for X, y_true in data_loader:

            X = X.to(device)
            X = X.float()
            y_true = y_true.to(device)

            _, y_prob = model(X)
            _, predicted_labels = torch.max(y_prob, 1)

            n += y_true.size(0)
            correct_pred += (predicted_labels == y_true).sum()

    return correct_pred.float() / n

# plot training vs. validation losses
def plot_losses(train_losses, valid_losses):
    '''
    Function for plotting training and validation losses
    '''
    
    # temporarily change the style of the plots to seaborn 
    plt.style.use('seaborn')

    train_losses = np.array(train_losses) 
    valid_losses = np.array(valid_losses)

    fig, ax = plt.subplots(figsize = (8, 4.5))

    ax.plot(train_losses, color='blue', label='Training loss') 
    ax.plot(valid_losses, color='red', label='Validation loss')
    ax.set(title="Loss over epochs", 
            xlabel='Epoch',
            ylabel='Loss') 
    ax.legend()
    fig.show()
    
    # change the plot style to default
    plt.style.use('default')

# model train function
def train(train_loader, model, criterion, optimizer, device):
    '''
    Function for the training step of the training loop
    '''

    model.train()
    running_loss = 0
    
    for X, y_true in train_loader:

        optimizer.zero_grad()
        
        X = X.to(device)
        X = X.float()
        y_true = y_true.to(device)
    
        # Forward pass
        y_hat, _ = model(X)
        #print(f'y_hat: {y_hat}, \ny_true: {y_true}') # FOR DEBUGGING
        loss = criterion(y_hat, y_true.long()) 
        running_loss += loss.item() * X.size(0)

        # Backward pass
        loss.backward()
        optimizer.step()
        
    epoch_loss = running_loss / len(train_loader.dataset)
    return model, optimizer, epoch_loss

# model validate function
def validate(valid_loader, model, criterion, device):
    '''
    Function for the validation step of the training loop
    '''
   
    model.eval()
    running_loss = 0
    
    for X, y_true in valid_loader:
    
        X = X.to(device)
        X = X.float()
        y_true = y_true.to(device)

        # Forward pass and record loss
        y_hat, _ = model(X)
        loss = criterion(y_hat, y_true.long()) 
        running_loss += loss.item() * X.size(0)

    epoch_loss = running_loss / len(valid_loader.dataset)
        
    return model, epoch_loss

# model training loop
def training_loop(model, criterion, optimizer, train_loader, valid_loader, epochs, device, print_every=1):
    '''
    Function defining the entire training loop
    '''
    
    # set objects for storing metrics
    best_loss = 1e10
    train_losses = []
    valid_losses = []
 
    # Train model
    for epoch in range(0, epochs):

        # training
        model, optimizer, train_loss = train(train_loader, model, criterion, optimizer, device)
        train_losses.append(train_loss)

        # validation
        with torch.no_grad():
            model, valid_loss = validate(valid_loader, model, criterion, device)
            valid_losses.append(valid_loss)

        if epoch % print_every == (print_every - 1):
            
            train_acc = get_accuracy(model, train_loader, device=device)
            valid_acc = get_accuracy(model, valid_loader, device=device)
                
            print(f'{datetime.now().time().replace(microsecond=0)} --- '
                  f'Epoch: {epoch}\t'
                  f'Train loss: {train_loss:.4f}\t'
                  f'Valid loss: {valid_loss:.4f}\t'
                  f'Train accuracy: {100 * train_acc:.2f}\t'
                  f'Valid accuracy: {100 * valid_acc:.2f}')

    plot_losses(train_losses, valid_losses)
    
    return model, optimizer, (train_losses, valid_losses)

### Load dataset

In [52]:
# define transforms
transforms = transforms.Compose([transforms.Resize((32, 32)),
                                 transforms.ToTensor()])

# download and create datasets
train_dataset = datasets.MNIST(root='mnist_data', 
                               train=True, 
                               transform=transforms,
                               download=True)

valid_dataset = datasets.MNIST(root='mnist_data', 
                               train=False, 
                               transform=transforms)

# define the data loaders
train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=BATCH_SIZE, 
                          shuffle=True)

valid_loader = DataLoader(dataset=valid_dataset, 
                          batch_size=BATCH_SIZE, 
                          shuffle=False)

### Define model

In [53]:
class LeNet5(nn.Module):

    def __init__(self, n_classes, preprocess=lambda x: x):
        super(LeNet5, self).__init__()

        self.preprocess = preprocess # Preprocess lambda ;)
        
        self.feature_extractor = nn.Sequential(            
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1),
            nn.Tanh()
        )

        self.classifier = nn.Sequential(
            nn.Linear(in_features=120, out_features=84),
            nn.Tanh(),
            nn.Linear(in_features=84, out_features=n_classes),
        )


    def forward(self, x):
        x = self.preprocess(x)
        x = self.feature_extractor(x)
        x = torch.flatten(x, 1)
        logits = self.classifier(x)
        probs = F.softmax(logits, dim=1)
        return logits, probs

In [54]:
torch.manual_seed(RANDOM_SEED)

model = LeNet5(N_CLASSES).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss()

In [55]:
#model, optimizer, _ = training_loop(model, criterion, optimizer, train_loader, valid_loader, N_EPOCHS, DEVICE)

# LeNET Applied to assignment data

### Load assignment data

In [56]:
import Data
from torch.utils.data import TensorDataset

# Dataset variables
VAL_SPLIT = 25000

# Partition labeled data into train and val datasets
X_tmp = Data.labeled_images#.reshape(-1, 3136)
y_tmp = Data.integer_labels() # Binary to number/letter, then number/letter to unique integer represntation for each label (conv net is only happy with 1D numerical outputs)
X_val, y_val = X_tmp[VAL_SPLIT:], y_tmp[VAL_SPLIT:]
X_train, y_train = X_tmp[0:VAL_SPLIT], y_tmp[0:VAL_SPLIT]

print(f'X_train: {X_train.shape} \ny_train: {y_train.shape}')
print(f'X_val: {X_val.shape} \ny_val: {y_val.shape}')

# Transform the labeled train and val datasets into pytorch Tensors
X_train, y_train, X_val, y_val = map(
    torch.tensor, (X_train, y_train, X_val, y_val)
)

# Initialize pytorch TensorDatasets
train_ds = TensorDataset(X_train, y_train)
val_ds = TensorDataset(X_val, y_val)

# Initialize pytorch DataLoaders
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True) # Shuffled to prevent correlation betwen batch ordering and model overfitting, see: https://pytorch.org/tutorials/beginner/nn_tutorial.html#add-validation
valid_loader = DataLoader(val_ds, batch_size=BATCH_SIZE)

X_train: (25000, 56, 56) 
y_train: (25000,)
X_val: (5000, 56, 56) 
y_val: (5000,)


### Train model on project dataset

In [59]:
torch.manual_seed(RANDOM_SEED)

# Set number of classes
N_CLASSES = 260 # One for each letter-number combo

def preprocess_batch(batch):
    batch = batch.view(-1, 1, 56, 56) # Resize input to make it square
    downsample = nn.AdaptiveAvgPool2d((32, 32)) # Define a downsampling function
    batch = downsample(batch) # Downsample the batch's image data
    return batch

model = LeNet5(N_CLASSES, preprocess=preprocess_batch).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss()

In [60]:
model, optimizer, _ = training_loop(model, criterion, optimizer, train_loader, valid_loader, N_EPOCHS, DEVICE)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x5880 and 120x84)